# 🚀 Hybrid Mamba-xLSTM: Google Colab Setup

Complete setup and training guide for Google Colab

## Step 1: Check GPU & Install Dependencies

In [ ]:
# Check GPU availability
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU available. Please enable GPU in Runtime settings!")

In [ ]:
# Clone and install the project and test it out 
!git clone https://github.com/krishankb-de/hybrid_model_mamba_xlstm.git
%cd hybrid_model_mamba_xlstm
!pip install -e . -q

## Step 2: Mount Google Drive (Optional, for saving checkpoints)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("✓ Google Drive mounted!")

## Step 3: Quick Test (2 minutes)

In [ ]:
import torch
from transformers import AutoTokenizer
import sys

sys.path.insert(0, '/content/hybrid_model_mamba_xlstm')
from hybrid_xmamba.models.configuration_hybrid import HybridConfig
from hybrid_xmamba.models.hybrid_lm import HybridLanguageModel

print('Loading model...')
config = HybridConfig(
    dim=1024,
    num_layers=24,
    vocab_size=50257,
    state_size=16,
    conv_size=4,
    expand_factor=2,
    use_fast_path=True,
    head_dim=64,
    num_heads=16,
    use_tfla=True,
    proj_factor=2,
    slstm_hidden_dim=1024,
    slstm_num_heads=4,
    use_exponential_gate=True,
)

model = HybridLanguageModel(config).eval().cuda()
tokenizer = AutoTokenizer.from_pretrained('gpt2')

print('Testing inference...')
text = 'The quick brown fox'
inputs = tokenizer(text, return_tensors='pt').to('cuda')

with torch.no_grad():
    outputs = model(**inputs)
    
print(f'✓ Model loaded and inference works!')
print(f'Output shape: {outputs.logits.shape}')
print(f'Model parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M')
print('✅ Quick test completed successfully!')

## Step 4: Full Training (Optional)

In [ ]:
# Full training (for local machine with GPU, not recommended for Colab free tier)
# Uncomment and run this on a machine with >= 24GB GPU VRAM
# Note: The 350M model is too large for Colab T4 (15GB). 
# For Colab training, consider using a smaller model or local machine.

# !python scripts/train.py \
#     model=hybrid_350m \
#     dataset=wikitext \
#     trainer=colab_single_gpu \
#     trainer.max_epochs=3 \
#     trainer.default_root_dir=/content/drive/MyDrive/hybrid_mamba_checkpoints \
#     dataset.batch_size=4 \
#     dataset.eval_batch_size=4 \
#     dataset.num_workers=0 \
#     +dataset.max_seq_length=128 \
#     wandb.enabled=false

print("📝 For full training on Colab, use a smaller model or local GPU")
print("✅ Step 3 (inference test) completed successfully!")
print("🚀 To train locally: python scripts/train.py model=hybrid_350m dataset=wikitext trainer=single_gpu")